# Adoption extraction tables

# Contents
1. [Import packages and options](#import_packages)
2. [Define key variables](#define_key_variables)
3. [Adoption_parties](#adoption_parties)
4. [Adoptions_applications](#adoptions_applications)
5. [Adoptions_cases](#adoptions_cases)
6. [Adoptions_disposals](#adoptions_disposals)
7. [Adoptions_disposal_fields](#adoptions_disposal_fields)

## Import packages and set options 
<a name="import_packages"></a>

In [ ]:
import pandas as pd  # for the data structures to store and manipulate tables
import pydbtools as pydb  # see https://github.com/moj-analytical-services/pydbtools
import boto3  # for working with AWS

# few things for viewing dataframes better
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

## Define some variables to be used throughout the notebook 
<a name="define_key_variables"></a>

In [ ]:
database = "familyman_dev_v2"
snapshot_date = "2021-08-19"
database_derived = "familyman_derived_dev_v1"

fcsq_database = "fcsq"

s3 = boto3.resource("s3")
bucket = s3.Bucket("alpha-family-data")

## Adoption_parties table
<a name="adoption_parties"></a>

### Drop the adoptions_parties table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adoptions_parties = f"""
DROP TABLE IF EXISTS fcsq.ADOPTIONS_PARTIES;
"""
pydb.read_sql_query(drop_adoptions_parties)

# clean up previous adoptions_parties files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adoptions_Parties/").delete()

### Create the adoptions_parties table in Athena

In [ ]:
create_adoptions_parties = f"""
CREATE TABLE IF NOT EXISTS fcsq.ADOPTIONS_PARTIES
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adoptions_Parties') AS
SELECT R.CASE_NUMBER,
  R.ROLE,
  R.ROLE_MODEL,
  R.REPRESENTATIVE_ROLE AS Representative,
  F.FIELD_MODEL,
  F.VALUE AS Rel_to_child,
  R.PARTY,
  P.DOB,
  P.GENDER,
  P.MOJAP_SNAPSHOT_DATE
FROM {database}.parties P
INNER JOIN {database}.roles R
ON P.PARTY = R.PARTY
INNER JOIN {database}.role_fields F
ON R.ROLE               = F.ROLE
WHERE ((R.ROLE_MODEL     = 'APLZ'
AND F.FIELD_MODEL = 'APLZ_RTC')
OR (R.ROLE_MODEL        = 'CHLDZ'
AND F.FIELD_MODEL = 'CHLDZ_CP'))
AND R.mojap_snapshot_date = date '{snapshot_date}'
AND F.mojap_snapshot_date = date '{snapshot_date}'
AND P.mojap_snapshot_date = date '{snapshot_date}'
"""

pydb.read_sql_query(create_adoptions_parties)



#### Adoption_parties validation

In [ ]:
# Query the table just created. The database to query is called __temp__, this is
# an alias for a sandbox database that is created for each user. For more details, see
# the pydbtools docs

parties_count = pydb.read_sql_query("select count(*) as count from fcsq.adoptions_parties")

parties_count

## Adoptions_applications table
<a name="adoptions_applications"></a>

### Drop the adoptions_applications table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adoptions_applications = "DROP TABLE IF EXISTS fcsq.ADOPTIONS_APPLICATIONS;"
pydb.read_sql_query(drop_adoptions_applications)

bucket.objects.filter(
    Prefix="fcsq_processing/Adoption/Adoptions_applications/"
).delete()

### Create the adoptions_applications table in Athena

In [ ]:
create_adoptions_applications_u21 = f"""
SELECT E.CASE_NUMBER,
  E.EVENT,
  E.RECEIPT_DATE,
  E.ENTRY_DATE,
  CASE E.RECEIPT_DATE
      WHEN NULL THEN E.ENTRY_DATE
      ELSE E.RECEIPT_DATE
  END AS Receipt_date2,
  E.EVENT_MODEL,
  F.VALUE  AS App_Type,
  date_parse(F1.VALUE, '%Y-%m-%d') AS Date_of_issue,
  F2.VALUE AS High_court
FROM {database}.events E
INNER JOIN {database}.event_fields F2
ON E.EVENT = F2.EVENT
INNER JOIN {database}.event_fields F1
ON E.EVENT = F1.EVENT
INNER JOIN {database}.event_fields F
ON E.EVENT               = F.EVENT
WHERE (E.EVENT_MODEL      = 'U21'
AND F.FIELD_MODEL  = 'U21_1'
AND F1.FIELD_MODEL = 'U21_2'
AND F2.FIELD_MODEL = 'U21_HC')
AND E.mojap_snapshot_date = date '{snapshot_date}'
AND F.mojap_snapshot_date = date '{snapshot_date}'
AND F1.mojap_snapshot_date = date '{snapshot_date}'
AND F2.mojap_snapshot_date = date '{snapshot_date}';
"""


pydb.create_temp_table(create_adoptions_applications_u21, "adoptions_applications_u21")
print("u21 applications done")

create_adoptions_applications_G50 = f"""
SELECT
  E.CASE_NUMBER,
  E.EVENT,
  E.RECEIPT_DATE,
  E.ENTRY_DATE,
  CASE E.RECEIPT_DATE
      WHEN NULL THEN E.ENTRY_DATE
      ELSE E.RECEIPT_DATE
  END AS Receipt_date2,
  E.EVENT_MODEL,
  CASE WHEN F.VALUE like '%RUK%' and F.VALUE like '%CCS%' THEN 'RUK,CCS'
       WHEN F.VALUE like '%RUK%' THEN 'RUK'
       WHEN F.VALUE like '%CCS%' THEN 'CCS'
        END AS App_Type, /*Have done it like this so that we do not include non adoption app types*/
  CASE E.RECEIPT_DATE
      WHEN NULL THEN E.ENTRY_DATE
      ELSE E.RECEIPT_DATE
  END AS Date_of_issue,
  'N' AS High_court
FROM
  {database}.events E
  INNER JOIN {database}.event_fields F
    ON E.EVENT = F.EVENT
WHERE
  F.FIELD_MODEL = 'G50_AT'
  AND (F.VALUE like '%RUK%'
   OR F.VALUE like '%CCS%')
   AND E.mojap_snapshot_date = date '{snapshot_date}'
   AND F.mojap_snapshot_date = date '{snapshot_date}';
"""

pydb.create_temp_table(create_adoptions_applications_G50, "adoptions_applications_G50")


print("G50 cases done")

create_adoptions_applications = f"""
CREATE TABLE IF NOT EXISTS fcsq.ADOPTIONS_APPLICATIONS
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adoptions_applications') AS
SELECT
  *
FROM
  __temp__.ADOPTIONS_APPLICATIONS_U21
WHERE case_number <> 'CV11Z00105'
UNION ALL
SELECT
  *
FROM
  __temp__.ADOPTIONS_APPLICATIONS_G50
WHERE case_number <> 'CV11Z00105';
"""

pydb.read_sql_query(create_adoptions_applications)

print("adoptions_applications table created")


#### Adoptions_applications validation

In [ ]:
applications_count = pydb.read_sql_query("select count(*) as count from fcsq.adoptions_applications")
applications_count

## Adoptions_cases table
<a name="adoptions_cases"></a>

### Drop the adoptions_cases table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adoptions_cases = "DROP TABLE IF EXISTS fcsq.ADOPTIONS_CASES;"
pydb.read_sql_query(drop_adoptions_cases)

bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adoptions_cases/").delete()

### Create the adoptions_cases table in Athena

In [ ]:
create_adoptions_cases = f"""
CREATE TABLE IF NOT EXISTS fcsq.ADOPTIONS_CASES
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adoptions_cases') AS
SELECT F.CASE_NUMBER,
  F.VALUE AS Contested,
  F1.VALUE AS Case_issue_date,
  F2.VALUE AS Case_app_type,
  CASE F.CASE_NUMBER
      WHEN 'BM08Z09028' THEN '2008-09-19'
      ELSE F1.VALUE
  END AS Case_issue_date2
FROM {database}.case_fields F
INNER JOIN {database}.case_fields F2
ON F.CASE_NUMBER = F2.CASE_NUMBER
INNER JOIN {database}.case_fields F1
ON F.CASE_NUMBER    = F1.CASE_NUMBER
WHERE F.FIELD_MODEL = 'FM3A_AC'
AND F1.FIELD_MODEL  = 'FM3A_DOI'
AND F2.FIELD_MODEL  = 'FM3A_AT'
AND F.case_number <> 'CV11Z00105'
AND F.mojap_snapshot_date = date '{snapshot_date}'
AND F1.mojap_snapshot_date = date '{snapshot_date}'
AND F2.mojap_snapshot_date = date '{snapshot_date}';

"""
pydb.read_sql_query(create_adoptions_cases)

#### Adoptions_cases validation

In [ ]:
cases_count = pydb.read_sql_query("select count(*) as count from fcsq.adoptions_cases")
cases_count

## Adoptions_disposals table
<a name="adoptions_disposals"></a>

### Drop the adoptions_disposals table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adoptions_disposals = "DROP TABLE IF EXISTS fcsq.ADOPTIONS_DISPOSALS;"
pydb.read_sql_query(drop_adoptions_disposals)

bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adoptions_disposals/").delete()

### Create the adoptions_disposals table in Athena

In [ ]:
create_adoptions_disposals = f"""
CREATE TABLE IF NOT EXISTS fcsq.ADOPTIONS_DISPOSALS
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adoptions_disposals') AS
SELECT E.CASE_NUMBER,
  E.EVENT,
  E.RECEIPT_DATE,
  E.ENTRY_DATE,
  CASE E.RECEIPT_DATE
      WHEN NULL THEN E.ENTRY_DATE
      ELSE E.RECEIPT_DATE  
  END AS Receipt_date2,
  E.EVENT_MODEL
FROM {database}.events E
WHERE ((E.EVENT_MODEL IN ('A70', 'A71', 'A72', 'A73', 'A74', 'A75', 'A76', 'A77', 'A78', 'A79', 'A80', 'A81', 'A12', 'A13', 'A15')
AND E.ERROR          = 'N')
OR (E.EVENT_MODEL    IN ('G63', 'ORDREF', 'ORDNOM')
AND E.ERROR          = 'N')
OR (E.EVENT_MODEL    IN ('A12', 'A13', 'A15')
AND E.ERROR          = 'N'))
AND E.case_number <> 'CV11Z00105'
AND E.mojap_snapshot_date = date '{snapshot_date}'; 
"""
pydb.read_sql_query(create_adoptions_disposals)

#### Adoptions_disposals validation

In [ ]:
disposals_count = pydb.read_sql_query("select count(*) as count from fcsq.adoptions_disposals")
disposals_count

## Adoptions_disposal_fields table
<a name="adoptions_disposal_fields"></a>

### Drop the adoptions_disposal_fields table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adoptions_disposal_fields = "DROP TABLE IF EXISTS fcsq.ADOPTIONS_DISPOSAL_FIELDS;"
pydb.read_sql_query(drop_adoptions_disposal_fields)

bucket.objects.filter(
    Prefix="fcsq_processing/Adoption/Adoptions_disposal_fields/"
).delete()

### Create the adoptions_disposal_fields table in Athena

In [ ]:
create_adoptions_disposal_fields = f"""
CREATE TABLE IF NOT EXISTS fcsq.ADOPTIONS_DISPOSAL_FIELDS
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adoptions_disposal_fields') AS
SELECT EVENT,
FIELD_MODEL,
VALUE
FROM {database}.event_fields E
WHERE (FIELD_MODEL LIKE 'A1%'
  OR FIELD_MODEL LIKE 'A7%'
  OR FIELD_MODEL IN ('G63_1', 'ORDNOM_5', 'ORDREF_5', 'A80_4', 'A81_5'))
  AND E.mojap_snapshot_date = date '{snapshot_date}';
"""
pydb.read_sql_query(create_adoptions_disposal_fields)

#### Adoptions_disposal_fields validation

In [ ]:
disposal_fields_count = pydb.read_sql_query("select count(*) as count from fcsq.adoptions_disposal_fields")
disposal_fields_count

# Stage 1 - Create The Main Parties Data

## Adopt_app_count table

### Drop the adopt_app_count table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adopt_app_count = f"""
DROP TABLE IF EXISTS fcsq.ADOPT_APP_COUNT;
"""
pydb.read_sql_query(drop_adopt_app_count)

# clean up previous adopt_app_count files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_app_count/").delete()

### Create the adopt_app_count table in Athena

In [ ]:
create_adopt_app_count = f"""
CREATE TABLE IF NOT EXISTS fcsq.ADOPT_APP_COUNT
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_app_count') AS
SELECT case_number, role_model, COUNT(role) AS countofrole
FROM fcsq.adoptions_parties
GROUP BY case_number, role_model
HAVING role_model = 'APLZ';
"""

pydb.read_sql_query(create_adopt_app_count)



#### adopt_app_count validation

In [ ]:
create_adopt_app_count_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_app_count")
create_adopt_app_count_count

## Adopt_appl_sex table

### Drop the adopt_appl_sex table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adopt_appl_sex = f"""
DROP TABLE IF EXISTS fcsq.ADOPT_APPL_SEX;
"""
pydb.read_sql_query(drop_adopt_appl_sex)

# clean up previous adopt_appl_sex files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_appl_sex/").delete()

### Create the adopt_appl_sex table in Athena

In [ ]:
create_adopt_appl_sex = f"""
CREATE TABLE IF NOT EXISTS fcsq.ADOPT_APPL_SEX
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_appl_sex') AS
SELECT app_count.case_number, app_count.role_model, 
app_count.countofrole, 
MIN(parties.gender) min_sex,
MAX(parties_1.gender) max_sex
FROM fcsq.adopt_app_count app_count INNER JOIN fcsq.adoptions_parties parties ON app_count.case_number = parties.case_number
INNER JOIN fcsq.adoptions_parties parties_1 ON app_count.case_number = parties_1.case_number
GROUP BY app_count.case_number, app_count.role_model, app_count.countofrole, parties.role_model, parties_1.role_model
HAVING parties.role_model='APLZ' AND parties_1.role_model='APLZ';
"""

pydb.read_sql_query(create_adopt_appl_sex)



#### adopt_appl_sex validation

In [ ]:
adopt_appl_sex_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_appl_sex")
adopt_appl_sex_count

## adopt_appl_rtc table

### Drop the adopt_appl_rtc table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adopt_appl_rtc = f"""
DROP TABLE IF EXISTS fcsq.adopt_appl_rtc;
"""
pydb.read_sql_query(drop_adopt_appl_rtc)

# clean up previous adopt_appl_rtc files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_appl_rtc/").delete()

### Create the adopt_appl_rtc table in Athena

In [ ]:
create_adopt_appl_rtc = f"""
CREATE TABLE IF NOT EXISTS fcsq.ADOPT_APPL_RTC
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_appl_rtc') AS
SELECT appl_sex.case_number, 
        appl_sex.role_model, 
        appl_sex.countofrole, 
        appl_sex.min_sex, 
        appl_sex.max_sex, 
        MIN(parties.rel_to_child) min_rtc, 
        MAX(parties_1.rel_to_child) max_rtc
FROM fcsq.adopt_appl_sex appl_sex
INNER JOIN fcsq.adoptions_parties parties 
        ON appl_sex.case_number = parties.case_number 
INNER JOIN fcsq.adoptions_parties parties_1 
        ON appl_sex.case_number = parties_1.case_number
GROUP BY appl_sex.case_number, 
          appl_sex.role_model, 
          appl_sex.countofrole, 
          appl_sex.min_sex, 
          appl_sex.max_sex, 
          parties.role_model, 
          parties_1.role_model
HAVING parties.role_model='APLZ' AND parties_1.role_model='APLZ';
"""

pydb.read_sql_query(create_adopt_appl_rtc)



#### adopt_appl_rtc validation

In [ ]:
adopt_appl_rtc_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_appl_rtc")
adopt_appl_rtc_count

## adopt_applicant_type table

### Drop the adopt_applicant_type table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adopt_applicant_type = f"""
DROP TABLE IF EXISTS fcsq.adopt_applicant_type;
"""
pydb.read_sql_query(drop_adopt_applicant_type)

# clean up previous adopt_applicant_type files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_applicant_type/").delete()

### Create the adopt_applicant_type table in Athena

In [ ]:
create_adopt_applicant_type_a = f"""
SELECT case_number, 
        role_model, 
        countofrole number_applicants, 
        min_sex, 
        max_sex, 
        min_rtc, 
        max_rtc,
CASE  WHEN countofrole=1 THEN 'single' 
      WHEN min_sex=1 AND max_sex =1 THEN 'same-sex couple' 
      WHEN min_sex=2 AND max_sex=2  THEN 'same-sex couple' 
      WHEN min_sex=1 AND max_sex =2 THEN 'mixed-sex couple'
                ELSE 'Other or not stated'       
END adopter_type
FROM fcsq.adopt_appl_rtc;
"""
pydb.create_temp_table(create_adopt_applicant_type_a, "adopt_applicant_type_a")

create_adopt_applicant_type_b = f"""
SELECT case_number, 
        role_model, 
        number_applicants, 
        min_sex, 
        max_sex, 
        min_rtc,
        max_rtc,
        adopter_type,
CASE WHEN (min_rtc Like 'Step%' OR max_rtc Like 'Step%') THEN 'Step parent'
      WHEN (min_rtc Like 'Natural%' OR max_rtc like 'Natural%') THEN 'Other or not stated'  
      WHEN number_applicants > 2 THEN 'Other or not stated'
      WHEN (min_rtc = 'Other' OR max_rtc = 'Other') THEN 'Other or not stated'
      WHEN (min_rtc IN ('Child','Adoption Agency','Central Authority','Home Office','Local Authority','Voluntary Organisation') AND max_rtc IN ('Child','Adoption Agency','Central Authority','Home Office','Local Authority','Voluntary Organisation')) THEN 'Other or not stated'
      WHEN (min_rtc IN ('Child','Adoption Agency','Central Authority','Home Office','Local Authority','Voluntary Organisation') OR max_rtc IN ('Child','Adoption Agency','Central Authority','Home Office','Local Authority','Voluntary Organisation')) THEN 'Sole applicant'
      WHEN number_applicants >1 THEN adopter_type 
      ELSE 'Sole applicant' 
      END adopter 
FROM __temp__.adopt_applicant_type_a;
"""

pydb.create_temp_table(create_adopt_applicant_type_b, "adopt_applicant_type_b")

create_adopt_applicant_type = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_applicant_type
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_applicant_type') AS
SELECT
case_number, 
  role_model, 
  number_applicants, 
  min_sex, 
  max_sex, 
  min_rtc, 
  max_rtc, 
  adopter_type,
CASE WHEN (min_rtc = 'Foster Carer' AND max_rtc = 'Sibling') THEN 'Other or not stated'
        WHEN (adopter_type = 'mixed-sex couple' AND min_rtc = 'Uncle' AND max_rtc = 'Uncle') THEN 'Other or not stated'
        ELSE adopter
        END adopter
FROM __temp__.adopt_applicant_type_b;
"""

pydb.read_sql_query(create_adopt_applicant_type)



#### adopt_applicant_type validation

In [ ]:
adopt_applicant_type_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_applicant_type")
adopt_applicant_type_count

## adopt_child_parties table

### Drop the adopt_child_parties table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adopt_child_parties = f"""
DROP TABLE IF EXISTS fcsq.adopt_child_parties;
"""
pydb.read_sql_query(drop_adopt_child_parties)

# clean up previous adopt_child_parties files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_child_parties/").delete()

### Create the adopt_child_parties table in Athena

In [ ]:
create_adopt_child_parties = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_child_parties
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_child_parties') AS
SELECT case_number, 
        role child_role, 
        party child_party, 
        dob, 
        gender child_sex
FROM fcsq.adoptions_parties
WHERE role_model='CHLDZ';
"""


"""
Note:
Removed dob TO_DATE as not supported and left dob as original timestamp

"""
pydb.read_sql_query(create_adopt_child_parties)



#### adopt_child_parties validation

In [ ]:
adopt_child_parties_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_child_parties")
adopt_child_parties_count

# Stage 2 - Applications

## adopt_application_4 table

### Drop the adopt_application_4 table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adopt_application_4 = f"""
DROP TABLE IF EXISTS fcsq.adopt_application_4;
"""
pydb.read_sql_query(drop_adopt_application_4)

# clean up previous adopt_application_4 files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_application_4/").delete()

### Create the adopt_application_4 table in Athena

#### adopt_application_4 validation

In [ ]:
adopt_application_4_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_application_4")
adopt_application_4_count

In [ ]:
create_adopt_application1 = f"""
SELECT 	a1.case_number, 
        a1.event, 
        a1.receipt_date2, 
        a1.app_type, 
        c1.case_app_type, 
        a1.date_of_issue, 
        date_parse(c1.case_issue_date2, '%Y-%m-%d') AS case_issue_date2,
        a1.high_court, 
        c1.contested, 
        b1.number_applicants, 
        b1.min_sex, 
        b1.max_sex, 
        b1.adopter_type, 
        b1.min_rtc,
        b1.max_rtc, 
        b1.adopter, 
        SUBSTR(cast(event as varchar(50)), 1,3) court, 
        ', '||  app_type ||  ',' app_type2,
	
        CASE WHEN a1.date_of_issue <= a1.receipt_date2 
                THEN a1.date_of_issue
              ELSE a1.receipt_date2
              END min_date,

        CASE WHEN a1.app_type='AO' OR a1.app_type='CA' OR a1.app_type='PFO' OR a1.app_type='AD' OR a1.app_type='PF' OR a1.app_type='SP' THEN 'Adoption'
              WHEN a1.app_type LIKE '%AO%' OR a1.app_type LIKE  '%CA%' OR a1.app_type LIKE '%PF%' OR a1.app_type LIKE '%AD%' OR a1.app_type LIKE '%SP%' THEN 'Adoption+other'
              ELSE 'Non-adoption'
              END adoption

FROM (fcsq.adoptions_applications a1 
LEFT JOIN fcsq.adopt_applicant_type b1 ON a1.case_number = b1.case_number) 
LEFT JOIN fcsq.adoptions_cases c1 ON a1.case_number = c1.case_number;
"""

pydb.create_temp_table(create_adopt_application1,"adopt_application1")

create_adopt_application2 = f"""
SELECT case_number, 
        event, 
        min_date, 
        EXTRACT(YEAR FROM min_date) year,

        CASE WHEN  min_date IS NULL THEN NULL
              WHEN EXTRACT(MONTH FROM min_date) between 1 and 3 THEN 1
              WHEN EXTRACT(MONTH from min_date) between 4 and 6 THEN 2
              WHEN EXTRACT(month from min_date) between 7 and 9 THEN 3
              WHEN EXTRACT(month from min_date) between 10 and 12 THEN 4
              END quarter,

        court, 
        app_type, 
        app_type2, 
        adoption, 
        case_app_type, 
        high_court, 
        contested, 
        number_applicants, 
        min_sex, 
        max_sex, 
        adopter_type, 
        min_rtc, 
        max_rtc, 
        adopter, 
        receipt_date2, 
        date_of_issue, 
        case_issue_date2,

        CASE WHEN app_type2 LIKE ', AO%' OR app_type2 LIKE ', AD%' OR app_type2 LIKE ', CA%' OR app_type2 LIKE ', PFO%' OR app_type2 LIKE ', PF%' THEN 'Adoption' 
              ELSE 'N'
              END adoption1 ,

        CASE WHEN app_type2 LIKE ', AO%' OR app_type2 LIKE ', AD%' OR app_type2 LIKE ', SP%' THEN 1
              ELSE 0
              END standard ,

        CASE WHEN app_type2 LIKE '%, CA%' OR app_type2 LIKE '%, CA%'
              THEN 1 
              ELSE 0
              END convention , 

        CASE WHEN app_type2 LIKE '%, PF%'
              THEN 1
                ELSE 0
                END foreign , 

        CASE WHEN app_type2 LIKE '%, PLA%' OR app_type2 LIKE ', FO%'
              THEN 1
              ELSE 0
              END placement , 

        CASE WHEN app_type2 LIKE '%, RPLA%'
              THEN 1
              ELSE 0
              END placement_revoke  , 


        CASE WHEN app_type2 LIKE '%, VPLA%'
              THEN 1
              ELSE 0
              END  placement_vary , 

        CASE WHEN app_type2 LIKE '%, CNO%'
              THEN 1
              ELSE 0
              END contact_s26 , 

        CASE WHEN app_type2 LIKE '%, RCNO%'
              THEN 1
              ELSE 0
              END contact_s26_revoke, 

        CASE WHEN app_type2 LIKE '%, VCNO%'
              THEN 1
              ELSE 0
              END contact_s26_vary , 

        CASE WHEN app_type2 LIKE '%, CCS%' 
              THEN 1
              ELSE 0
              END change_surname , 

        CASE WHEN app_type2 LIKE '%, RUK%'
              THEN 1
              ELSE 0
              END  remove_child_from_uk, 

        CASE WHEN app_type2 LIKE '%, OR%'
              THEN 1
              ELSE 0
              END other_recovery,

        CASE WHEN app_type2 LIKE '%, PT10%'
              THEN 1
              ELSE 0
              END other_part_10, 

        CASE WHEN app_type2 LIKE '%, PT9%'
              THEN 1
              ELSE 0
              END other_part_9, 

        CASE WHEN app_type2 LIKE '%, S84%'
              THEN 1
              ELSE 0
              END other_s84_order, 

        CASE WHEN app_type2 LIKE '%, S88%'
              THEN 1
              ELSE 0
              END other_s88_direction , 

        CASE WHEN app_type2 LIKE '%, S89%'
              THEN 1
              ELSE 0
              END other_s89_order 

FROM __temp__.adopt_application1;
"""

pydb.create_temp_table(create_adopt_application2,"adopt_application2")

create_adopt_application3 = f"""
SELECT application2.case_number, 
        MIN(application2.min_date) app_date, 
        application2.year, 
        application2.quarter, 
        application2.court, 
        application2.app_type, 
        application2.app_type2, 
        application2.case_app_type, 
        application2.adoption, 
        application2.high_court, 
        application2.contested, 
        application2.number_applicants, 
        application2.min_sex, 
        application2.max_sex, 
        application2.adopter_type, 
        application2.min_rtc, 
        application2.max_rtc, 
        application2.adopter
FROM __temp__.adopt_application2 application2
GROUP BY application2.case_number, 
          application2.year, 
          application2.quarter, 
          application2.court, 
          application2.app_type, 
          application2.app_type2, 
          application2.case_app_type, 
          application2.adoption, 
          application2.high_court, 
          application2.contested, 
          application2.number_applicants, 
          application2.min_sex, 
          application2.max_sex, 
          application2.adopter_type, 
          application2.min_rtc, 
          application2.max_rtc, 
          application2.adopter
HAVING (application2.case_number)<>'CV11Z00105';
"""

pydb.create_temp_table(create_adopt_application3,"adopt_application3")

create_adopt_application4 = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_application_4
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_application_4') AS
SELECT application2.case_number, 
        application2.min_date app_date, 
        application2.year, 
        application2.quarter, 
        application2.court, 
        application2.app_type, 
        application2.case_app_type, 
        application2.adoption, 
        application2.high_court, 
        application2.contested, 
        application2.number_applicants, 
        application2.min_sex, 
        application2.max_sex, 
        application2.adopter_type, 
        application2.min_rtc, 
        application2.max_rtc, 
        application2.adopter, 
        application2.standard, 
        application2.convention, 
        application2.foreign, 
        application2.placement, 

        application2.placement_revoke + application2.placement_vary  placement_revoke_or_vary, 

        application2.contact_s26, 

        application2.contact_s26_revoke + application2.contact_s26_vary contact_s26_revoke_or_vary, 
        
        application2.change_surname, 
        application2.remove_child_from_uk, 

        application2.other_recovery+application2.other_part_10+application2.other_part_9+application2.other_s84_order+application2.other_s88_direction+application2.other_s89_order other_order_type


FROM __temp__.adopt_application2 application2;
"""
pydb.read_sql_query(create_adopt_application4)

## adopt_application_5 table

### Drop the adopt_application_5 table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adopt_application_5 = f"""
DROP TABLE IF EXISTS fcsq.adopt_application_5;
"""
pydb.read_sql_query(drop_adopt_application_5)

# clean up previous adopt_application_5 files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_application_5/").delete()

### Create the adopt_application_5 table in Athena

In [ ]:
create_adopt_application_5 = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_application_5
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_application_5') AS
SELECT application4.*, 

        CASE WHEN adoption='Adoption' OR adoption='Adoption+other' 
              THEN 'Adoption'
              ELSE ' '
              END AS adoption_cases, 

        CASE WHEN adoption='Non-adoption' OR adoption='Adoption+other'
              THEN 'Non-adoption'
              ELSE '' 
              END AS non_adoption_cases, 

        CASE WHEN adopter IS NULL
              THEN 'Other or not stated'
              ELSE adopter
              END AS adopter_2

FROM fcsq.adopt_application4 application4;
"""

pydb.read_sql_query(create_adopt_application_5)



#### adopt_application_5 validation

In [ ]:
adopt_application_5_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_application_5")
adopt_application_5_count

## adopt_apps_6_adoptions_only table

### Drop the adopt_apps_6_adoptions_only table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adopt_apps_6_adoptions_only = f"""
DROP TABLE IF EXISTS fcsq.adopt_apps_6_adoptions_only;
"""
pydb.read_sql_query(drop_adopt_apps_6_adoptions_only)

# clean up previous adopt_apps_6_adoptions_only files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_apps_6_adoptions_only/").delete()

### Create the adopt_apps_6_adoptions_only table in Athena

In [ ]:
create_adopt_apps_6_adoptions_only = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_apps_6_adoptions_only
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_apps_6_adoptions_only')
AS
SELECT application_5.case_number, 
        application_5.app_date, 
        application_5.year, 
        application_5.quarter,
        application_5.court, 
        application_5.app_type, 
        application_5.case_app_type, 
        application_5.adoption, 
        application_5.high_court, 
        application_5.contested, 
        application_5.number_applicants, 
        application_5.min_sex, 
        application_5.max_sex, 
        application_5.adopter_type,
        application_5.min_rtc, 
        application_5.max_rtc, 
        application_5.adopter, 
        application_5.standard, 
        application_5.convention, 
        application_5.foreign,
        application_5.adoption_cases, 
        application_5.adopter_2,
        
        (application_5.standard + application_5.convention + application_5.foreign) AS adoptions_total 
        
FROM fcsq.adopt_application_5 application_5
WHERE application_5.adoption_cases='Adoption';
"""

pydb.read_sql_query(create_adopt_apps_6_adoptions_only)



#### adopt_apps_6_adoptions_only validation

In [ ]:
adopt_apps_6_adoptions_only_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_apps_6_adoptions_only")
adopt_apps_6_adoptions_only_count

## adopt_apps_6_non_adoptions table

### Drop the adopt_apps_6_non_adoptions table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adopt_apps_6_non_adoptions = f"""
DROP TABLE IF EXISTS fcsq.adopt_apps_6_non_adoptions;
"""
pydb.read_sql_query(drop_adopt_apps_6_non_adoptions)

# clean up previous adopt_apps_6_non_adoptions files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_apps_6_non_adoptions/").delete()

### Create the adopt_apps_6_non_adoptions table in Athena

In [ ]:
create_adopt_apps_6_non_adoptions = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_apps_6_non_adoptions
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_apps_6_non_adoptions') AS
SELECT application_5.case_number, 
        application_5.app_date, 
        application_5.year, 
        application_5.quarter, 
        application_5.court, 
        application_5.app_type, 
        application_5.case_app_type, 
        application_5.adoption, 
        application_5.high_court, 
        application_5.contested, 
        application_5.number_applicants, 
        application_5.min_sex, 
        application_5.max_sex, 
        application_5.adopter_type, 
        application_5.min_rtc, 
        application_5.max_rtc, 
        application_5.adopter, 
        application_5.placement, 
        application_5.placement_revoke_or_vary, 
        application_5.contact_s26, 
        application_5.contact_s26_revoke_or_vary, 
        application_5.change_surname, 
        application_5.remove_child_from_uk, 
        application_5.other_order_type, 
        application_5.non_adoption_cases, 
        application_5.adopter_2, 
        
        placement+placement_revoke_or_vary+contact_s26+contact_s26_revoke_or_vary+change_surname+remove_child_from_uk+other_order_type AS non_adoptions_total,
        
        REPLACE(app_type,'AO, ','') AS remove_ao
        
FROM fcsq.adopt_application_5 application_5
WHERE application_5.non_adoption_cases='Non-adoption' AND 
(placement+placement_revoke_or_vary+contact_s26+contact_s26_revoke_or_vary+change_surname+remove_child_from_uk+other_order_type)>0;
"""

pydb.read_sql_query(create_adopt_apps_6_non_adoptions)



#### adopt_apps_6_non_adoptions validation

In [ ]:
adopt_apps_6_non_adoptions_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_apps_6_non_adoptions")
adopt_apps_6_non_adoptions_count

## Summary 1: Adoption applications by adopter

In [ ]:
distinct_adopter_2 = f"""
SELECT DISTINCT adopter_2  FROM fcsq.adopt_apps_6_adoptions_only;
"""
table1 = pydb.read_sql_query(distinct_adopter_2)
table1

In [ ]:
sum_adopter_types = f"""
SELECT year, 
        quarter,
        SUM(CASE WHEN adopter_2='Other or not stated' THEN adoptions_total ELSE 0 END) other_or_not_stated,
        SUM(CASE WHEN adopter_2='Sole applicant' THEN adoptions_total ELSE 0 END) Sole_applicant,
        SUM(CASE WHEN adopter_2='Step parent' THEN adoptions_total ELSE 0  END) Step_Parent,
        SUM(CASE WHEN adopter_2='mixed-sex couple' THEN adoptions_total ELSE 0 END) mix_sex_couple,
        SUM(CASE WHEN adopter_2='same-sex couple' THEN adoptions_total ELSE 0 END) same_sex_couple,
        SUM(adoptions_total)
FROM fcsq.adopt_apps_6_adoptions_only WHERE year > 2010 
GROUP BY year, 
          quarter
ORDER BY 1,2;
"""
table1 = pydb.read_sql_query(sum_adopter_types)
table1

## Summary 2: non-adoption applications by adopter

In [ ]:
adopt_non_adopt_summary = f"""
SELECT year, 
        quarter,
        SUM (placement) AS placement_total,
        SUM (placement_revoke_or_vary) AS placement_revoke_or_vary,
        SUM (contact_s26 ) AS contact_s26,
        SUM (contact_s26_revoke_or_vary) AS contact_s26_revoke_or_vary,
        SUM (change_surname)AS change_surname,
        SUM (remove_child_from_uk) AS remove_child_from_uk,
        SUM (other_order_type)AS other_order_type,
        SUM (non_adoptions_total) AS non_adoptions_total_sum

FROM fcsq.adopt_apps_6_non_adoptions 
WHERE year > 2010 /*RB Note (18/11/15): Added this qualifier in as presently results before 2010 not used.*/
GROUP BY year, 
          quarter
ORDER BY year,
          quarter;
"""

table1 = pydb.read_sql_query(adopt_non_adopt_summary)
table1

## Summary 3 - produces a case level table for purposes of case level counts

In [ ]:
adopt_application_case_count_a = f"""
SELECT case_number,
        court, 
        MIN(app_date) AS minofapp_date
FROM fcsq.adopt_application4 
WHERE year > 2010
GROUP BY case_number,court
ORDER BY 1,2;
"""
pydb.create_temp_table(adopt_application_case_count_a,"adopt_application_case_count_a")

In [ ]:
adopt_application_case_count_b = f"""
SELECT case_number, 
        minofapp_date, 
        CAST(EXTRACT(YEAR FROM minofapp_date) as varchar(3)) || ' - ' ||
        
        (CASE WHEN (EXTRACT (MONTH FROM minofapp_date)) BETWEEN 1 AND 3  THEN CAST(1 as varchar(3))
              WHEN (EXTRACT (MONTH FROM minofapp_date)) BETWEEN 4 AND 6 THEN CAST(2 as varchar(3))
              WHEN (EXTRACT (MONTH FROM minofapp_date)) BETWEEN 7 AND 9 THEN CAST(3 as varchar(3))
              WHEN (EXTRACT (MONTH FROM minofapp_date)) BETWEEN 10 AND 12 THEN CAST(4 as varchar(3))
              END) quarter

FROM __temp__.adopt_application_case_count_a;
"""
pydb.create_temp_table(adopt_application_case_count_b,"adopt_application_case_count_b")


In [ ]:
adopt_apps_case_count_total = f"""
SELECT quarter, 
        COUNT(case_number) count_of_case
FROM __temp__.adopt_application_case_count_b
GROUP BY quarter
ORDER BY quarter; 
"""


pydb.create_temp_table(adopt_apps_case_count_total,"adopt_apps_case_count_total")

In [ ]:
table1 = pydb.read_sql_query("select * from __temp__.adopt_application_case_count_a")
table1

In [ ]:
table2 = pydb.read_sql_query("select * from __temp__.adopt_application_case_count_b")
table2

In [ ]:
table3 = pydb.read_sql_query("select * from __temp__.adopt_apps_case_count_total")
table3

## adopt_adopt_only_case_count summary

In [ ]:
adopt_adopt_only_case_count_a = f"""
SELECT case_number, 
        MIN(app_date) AS minofapp_date 
FROM fcsq.adopt_apps_6_adoptions_only
WHERE year > 2010
GROUP BY case_number, court;
"""
pydb.create_temp_table(adopt_adopt_only_case_count_a,"adopt_adopt_only_case_count_a")

adopt_adopt_only_case_count_b = f"""
SELECT case_number, 
        minofapp_date, 
        CAST(EXTRACT(YEAR FROM (minofapp_date)) as varchar(3)) || ' - ' ||

        (CASE WHEN (EXTRACT (MONTH FROM minofapp_date)) BETWEEN 1 AND 3  THEN CAST(1 as varchar(3))
              WHEN (EXTRACT (MONTH FROM minofapp_date)) BETWEEN 4 AND 6 THEN CAST(2 as varchar(3))
              WHEN (EXTRACT (MONTH FROM minofapp_date)) BETWEEN 7 AND 9 THEN CAST(3 as varchar(3))
              WHEN (EXTRACT (MONTH FROM minofapp_date)) BETWEEN 10 AND 12 THEN CAST(4 as varchar(3))
              END) quarter

FROM __temp__.adopt_adopt_only_case_count_a;
"""
pydb.create_temp_table(adopt_adopt_only_case_count_b,"adopt_adopt_only_case_count_b")

adopt_only_case_count_total = f"""
SELECT quarter, 
        COUNT (case_number) AS count_of_case
FROM __temp__.adopt_adopt_only_case_count_b
GROUP BY quarter
ORDER BY quarter;
"""
pydb.create_temp_table(adopt_only_case_count_total,"adopt_adopt_only_case_count_total")

## The next couple of queries are here to find the court level data for the court level/DFJ CSV file. Currently these are not used for FCSQ

In [ ]:
applications_court_level = f"""
SELECT year, 
        quarter, 
        court, 
        COUNT(*) n 
FROM fcsq.adopt_apps_6_adoptions_only
WHERE year>2010
GROUP BY year, 
          quarter, 
          court
ORDER BY 1,2,3;
"""

pydb.read_sql_query(applications_court_level)

In [ ]:
#Secondly the applications case level court level file
adopt_application_case_count_c = f"""
SELECT 
  case_number, 
  MIN(app_date) AS minofapp_date,
  court 
FROM fcsq.adopt_apps_6_adoptions_only 
GROUP BY case_number,
          court;
"""

pydb.create_temp_table(adopt_application_case_count_c,"adopt_application_case_count_c")

adopt_application_case_count_d = f"""
SELECT case_number, 
        EXTRACT(YEAR FROM (minofapp_date)) year,
        CAST(EXTRACT(YEAR FROM (minofapp_date)) as varchar(3)) || ' - ' ||

        (CASE WHEN (EXTRACT (MONTH FROM minofapp_date)) BETWEEN 1 AND 3  THEN CAST(1 as varchar(3))
              WHEN (EXTRACT (MONTH FROM minofapp_date)) BETWEEN 4 AND 6 THEN CAST(2 as varchar(3))
              WHEN (EXTRACT (MONTH FROM minofapp_date)) BETWEEN 7 AND 9 THEN CAST(3 as varchar(3))
              WHEN (EXTRACT (MONTH FROM minofapp_date)) BETWEEN 10 AND 12 THEN CAST(4 as varchar(3))
              END) quarter,
        minofapp_date,
        court 
FROM __temp__.adopt_application_case_count_c;
"""

pydb.create_temp_table(adopt_application_case_count_c,"adopt_application_case_count_c")
pydb.create_temp_table(adopt_application_case_count_d,"adopt_application_case_count_d")


final_query = f"""
SELECT year, 
        quarter, 
        court, 
        COUNT(*) n 
FROM __temp__.adopt_application_case_count_d
WHERE year > 2010
GROUP BY year, 
          quarter, 
          court
ORDER BY year, 
          quarter, 
          court;
"""

pydb.read_sql_query(final_query)

# Stage 3 - Disposals

## adopt_order_type table

In [ ]:
#Create new disposal_fields table while old one not working
data = pd.read_csv('s3://alpha-family-data/fcsq_processing/Adoption/disposal_fields_csv/ADOPTIONS_DISPOSAL_FIELDS.csv')

import awswrangler as wr
wr.s3.to_parquet(  # Storing the data and metadata to Data Lake
    df=data,
    path="s3://alpha-family-data/fcsq_processing/Adoption/disposal_fields_csv/fields_parquet/",
    dataset=True, database = "default", table = "temporary_df"
)



### Drop the adopt_order_type table if it already exists and remove its data from the S3 bucket

In [ ]:

drop_adopt_order_type = f"""
DROP TABLE IF EXISTS fcsq.adopt_order_type;
"""
pydb.read_sql_query(drop_adopt_order_type)

# clean up previous adopt_order_type files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_order_type/").delete()

### Create the adopt_order_type table in Athena

In [ ]:
create_adopt_order_type = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_order_type
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_order_type') AS
SELECT event, 
        field_model, 
        value 
FROM fcsq.adoptions_disposal_fields_temporary
WHERE field_model='A73_1'
  OR field_model='A74_2' 
  OR field_model='A80_4' 
  OR field_model='G63_1' 
  OR field_model='ORDNOM_5' 
  OR field_model='ORDREF_5' 
  OR field_model='A81_5';
"""

pydb.read_sql_query(create_adopt_order_type)



#### adopt_order_type validation

In [ ]:
adopt_order_type_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_order_type")
adopt_order_type_count

## adopt_disposals_type table

### Drop the adopt_disposals_type table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adopt_disposals_type = f"""
DROP TABLE IF EXISTS fcsq.adopt_disposals_type;
"""
pydb.read_sql_query(drop_adopt_disposals_type)

# clean up previous adopt_disposals_type files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_disposals_type/").delete()

### Create the adopt_disposals_type table in Athena

In [ ]:
create_adopt_disposals_type = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_disposals_type
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_disposals_type') AS

SELECT disposals.case_number, 
        disposals.event, 
        disposals.receipt_date2, 
        disposals.event_model, 
        disposal_fields.field_model, 
        disposal_fields.value AS order_type, 
        EXTRACT(YEAR FROM (disposals.receipt_date2)) AS year, 

        CASE WHEN EXTRACT(MONTH FROM (disposals.receipt_date2))<4 THEN 1
              WHEN EXTRACT(MONTH FROM(disposals.receipt_date2)) <7 THEN 2
              WHEN EXTRACT( MONTH FROM (disposals.receipt_date2))<10 THEN 3
              ELSE 4
              END AS quarter

FROM fcsq.adoptions_disposals disposals 
  LEFT JOIN fcsq.ADOPTIONS_DISPOSAL_FIELDS_TEMPORARY disposal_fields 
  ON disposals.event = disposal_fields.event

WHERE (((disposals.event_model)='A70' 
    OR (disposals.event_model)='A71' 
    OR (disposals.event_model)='A72' 
    OR (disposals.event_model)='A75' 
    OR (disposals.event_model)='A79') 
    AND ((disposal_fields.field_model) LIKE '%_JG')) 
  OR (((disposals.event_model)='A76') 
    AND ((disposal_fields.field_model)='A76_12')) 
  OR (((disposals.event_model)='A77') 
    AND ((disposal_fields.field_model)='A77_12')) 
  OR (((disposals.event_model)='A78') 
    AND ((disposal_fields.field_model)='A78_15')) 
  OR (((disposals.event_model)='A12' 
    OR (disposals.event_model)='A13' 
    OR (disposals.event_model)='A15') 
    AND ((disposal_fields.field_model) LIKE '%_FO')) 
  OR (((disposals.event_model)='A73' 
    OR (disposals.event_model)='G63') 
    AND ((disposal_fields.field_model) LIKE '%_1')) 
  OR (((disposals.event_model)='A74') 
    AND ((disposal_fields.field_model)='A74_2')) 
  OR (((disposals.event_model)='A80') 
    AND ((disposal_fields.field_model)='A80_4')) 
  OR (((disposals.event_model)='ORDNOM' 
    OR (disposals.event_model)='ORDREF') 
    AND ((disposal_fields.field_model) LIKE '%_5'));
"""

pydb.read_sql_query(create_adopt_disposals_type)



#### adopt_disposals_type validation

In [ ]:
adopt_disposals_type_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_disposals_type")
adopt_disposals_type_count

## adopt_country_of_birth table

### Drop the adopt_country_of_birth table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adopt_country_of_birth = f"""
DROP TABLE IF EXISTS fcsq.adopt_country_of_birth;
"""
pydb.read_sql_query(drop_adopt_country_of_birth)

# clean up previous adopt_country_of_birth files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_country_of_birth/").delete()

### Create the adopt_country_of_birth table in Athena

In [ ]:
create_adopt_country_of_birth = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_country_of_birth
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_country_of_birth') AS
SELECT disposal_fields.event, 
        disposal_fields.field_model, 
        disposal_fields.value
FROM fcsq.ADOPTIONS_DISPOSAL_FIELDS_TEMPORARY disposal_fields
WHERE (((disposal_fields.field_model)='A70_5' 
    OR (disposal_fields.field_model)='A76_2' 
    OR (disposal_fields.field_model)='A77_10' 
    OR (disposal_fields.field_model)='A78_13' 
    OR ((disposal_fields.field_model)='A12_1' 
    OR (disposal_fields.field_model)='A15_1')));
"""

pydb.read_sql_query(create_adopt_country_of_birth)



#### adopt_country_of_birth validation

In [ ]:
adopt_country_of_birth_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_country_of_birth")
adopt_country_of_birth_count

# NOTE: LEFT OUT AGE FIX

## adopt_disposals_with_child table

### Drop the adopt_disposals_with_child table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adopt_disposals_with_child = f"""
DROP TABLE IF EXISTS fcsq.adopt_disposals_with_child;
"""
pydb.read_sql_query(drop_adopt_disposals_with_child)

# clean up previous adopt_disposals_with_child files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_disposals_with_child/").delete()

### Create the adopt_disposals_with_child table in Athena

In [ ]:
create_adopt_disposals2 =f"""
SELECT disposals1.case_number, 
        disposals1.event, 
        disposals1.receipt_date2, 
        disposals1.event_model, 
        order_type.field_model, 
        order_type.value AS order_type, 
        country_of_birth.value AS country_of_birth, 
        EXTRACT(YEAR FROM (disposals1.receipt_date2)) AS year, 

        CASE WHEN EXTRACT(Month FROM (disposals1.receipt_date2)) <4 THEN 1
              WHEN EXTRACT(Month FROM (disposals1.receipt_date2)) <7 THEN 2
              WHEN EXTRACT(Month FROM (disposals1.receipt_date2))<10 THEN 3
              ELSE 4
              END AS quarter
              
FROM fcsq.adoptions_disposals disposals1 
  LEFT JOIN fcsq.adopt_order_type order_type 
    ON disposals1.event = Order_type.event 
  LEFT JOIN fcsq.adopt_country_of_birth country_of_birth 
    ON disposals1.event = country_of_birth.event
WHERE (((disposals1.event_model)<>'G63' 
    AND (disposals1.event_model)<>'ORDNOM' 
    AND (disposals1.event_model)<>'ORDREF')) 
  OR (((disposals1.case_number) LIKE '____A%' 
      OR (disposals1.case_number) LIKE '____Z%') 
    AND ((disposals1.event_model)='G63' 
      OR (disposals1.event_model)='ORDNOM' 
      OR (disposals1.event_model)='ORDREF'));
"""

pydb.create_temp_table(adopt_disposals2,"adopt_disposals2")


create_adopt_disposals3 = f"""
SELECT disposals2.case_number, 
        disposals2.year, 
        disposals2.quarter, 
        disposals2.event, 
        disposals2.receipt_date2, 
        disposals2.event_model, 
        disposals2.field_model, 
        disposals2.order_type, 
        lower(disposals2.country_of_birth) as country_of_birth, 
        applicant_type.number_applicants,
        applicant_type.min_sex, 
        applicant_type.max_sex, 
        applicant_type.adopter_type, 
        applicant_type.min_rtc, 
        applicant_type.max_rtc, 
        applicant_type.adopter 
        
FROM __temp__.adopt_disposals2 disposals2 
  LEFT JOIN fcsq.adopt_applicant_type applicant_type
    ON disposals2.case_number = applicant_type.case_number;
"""
pydb.create_temp_table(create_adopt_disposals3,"adopt_disposals3")

create_adopt_disposals4 = f"""
SELECT disposals3.case_number, 
        substr(CAST(disposals3.event as varchar(3)),1,3) AS court, 
        disposals3.year, 
        disposals3.quarter,
        MIN(disposals3.receipt_date2) AS receipt_date, 
        disposals3.event_model, 
        disposals3.field_model, 
        disposals3.order_type, 
        MAX(disposals3.country_of_birth) AS country_of_birth_new,
        disposals3.number_applicants,
        disposals3.min_sex, 
        disposals3.max_sex, 
        disposals3.adopter_type, 
        disposals3.min_rtc, 
        disposals3.max_rtc, 
        disposals3.adopter
        
FROM __temp__.adopt_disposals3 disposals3
GROUP BY disposals3.case_number, 
          substr(CAST(disposals3.event as varchar(3)),1,3), 
          disposals3.year, 
          disposals3.quarter, 
          disposals3.event_model, 
          disposals3.field_model,
          disposals3.order_type, 
          disposals3.number_applicants, 
          disposals3.min_sex, 
          disposals3.max_sex, 
          disposals3.adopter_type, 
          disposals3.min_rtc, 
          disposals3.max_rtc, 
          disposals3.adopter
HAVING (((disposals3.case_number)<>'CV11Z00105'))
ORDER BY disposals3.case_number;
"""

pydb.create_temp_table(create_adopt_disposals4,"adopt_disposals4")

create_adopt_disposals4a = f"""
SELECT disposals4.case_number, 
        MIN(disposals4.court)AS court_new, 
        disposals4.year, 
        disposals4.quarter,
        disposals4.receipt_date, 
        disposals4.event_model, 
        disposals4.field_model, 
        disposals4.order_type, 
        disposals4.country_of_birth_new,
        disposals4.number_applicants,
        disposals4.min_sex, 
        disposals4.max_sex, 
        disposals4.adopter_type, 
        disposals4.min_rtc, 
        disposals4.max_rtc, 
        disposals4.adopter
        
FROM __temp__.adopt_disposals4 disposals4
GROUP BY disposals4.case_number,  
        disposals4.year, 
        disposals4.quarter,
        disposals4.receipt_date, 
        disposals4.event_model, 
        disposals4.field_model, 
        disposals4.order_type, 
        disposals4.country_of_birth_new,
        disposals4.number_applicants,
        disposals4.min_sex, 
        disposals4.max_sex, 
        disposals4.adopter_type, 
        disposals4.min_rtc, 
        disposals4.max_rtc, 
        disposals4.adopter
ORDER BY disposals4.case_number;
"""

pydb.create_temp_table(create_adopt_disposals4a,"adopt_disposals4a")

create_adopt_disposals_with_child = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_disposals_with_child
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_disposals_with_child') AS
SELECT disposals4a.*, 
        child_parties.dob, 
        child_parties.child_sex, 
        date_diff('day',dob,receipt_date)/365.25 AS child_age 
FROM __temp__.adopt_disposals4a Disposals4a
  LEFT JOIN fcsq.adopt_child_parties child_parties
    ON disposals4a.case_number = child_parties.case_number;
"""

pydb.read_sql_query(create_adopt_disposals_with_child)




#### adopt_disposals_with_child validation

In [ ]:
adopt_disposals_with_child_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_disposals_with_child")
adopt_disposals_with_child_count

## adopt_disposals5 table

### Drop the adopt_disposals5 table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adopt_disposals5 = f"""
DROP TABLE IF EXISTS fcsq.adopt_disposals5;
"""
pydb.read_sql_query(drop_adopt_disposals5)

# clean up previous adopt_disposals5 files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_disposals5/").delete()

### Create the adopt_disposals5 table in Athena

In [ ]:
create_adopt_disposals5 = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_disposals5
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_disposals5') AS
SELECT disposals_with_child.case_number, 
        disposals_with_child.court_new AS court, 
        disposals_with_child.year, 
        disposals_with_child.quarter, 
        disposals_with_child.receipt_date, 
        disposals_with_child.event_model, 
        disposals_with_child.field_model, 
        disposals_with_child.order_type, 
        disposals_with_child.country_of_birth_new AS country_of_birth, 
        disposals_with_child.number_applicants, 
        disposals_with_child.adopter_type, 
        disposals_with_child.adopter,
        max_rtc, 
        max_sex, 
        min_rtc, 
        min_sex, 
        child_sex,

        CASE WHEN disposals_with_child.child_age IS NULL THEN 'Unknown'
              WHEN disposals_with_child.child_age<0 THEN 'Other'
              WHEN disposals_with_child.child_age<1 THEN '<1 year'
              WHEN disposals_with_child.child_age<5 THEN '1-4 years'
              WHEN disposals_with_child.child_age<10 THEN'5-9 years'
              WHEN disposals_with_child.child_age<15 THEN '10-14 years'
              WHEN disposals_with_child.child_age<18 THEN '15-17 years'
              ELSE 'Other'
              END AS age_band, 

        disposals_with_child.child_age, 

        CASE WHEN event_model='A15' 
                OR event_model='A76' 
                OR event_model='A77' 
                THEN 'Adoption'
              WHEN event_model='G63' 
                OR event_model='ORDREF' 
                OR event_model='ORDNOM' 
                THEN 'No order made'
              ELSE 'Non-adoption'
              END  AS adoption, 

        CASE WHEN event_model='A77' 	
                THEN 'Convention'
              WHEN country_of_birth_new <>'' 
                And event_model='A76' 
                THEN '?Foreign?'
              WHEN event_model='A76' 
                OR event_model='A15' 
                THEN 'Standard'
              WHEN event_model='A12' 
                OR event_model='A70' 
                THEN 'Placement'
              WHEN event_model='A13' 
                OR event_model='A71' 
                OR event_model='A72' 
                THEN 'Placement_revoke_or_vary'
              WHEN order_type='CNO' 
                THEN 'Contact_s26'
              WHEN order_type='RCNO' 
                OR order_type='VCNO' 
                OR order_type='OFNC' 
                THEN 'Contact_s26_revoke_or_vary'
              WHEN order_type='CCS' 
                THEN 'Change_surname'
              WHEN order_type='RUK' 
                THEN 'Remove_child_from_UK'
              WHEN event_model='A75' 
                OR event_model='A78' 
                OR event_model='A79' 
                OR event_model='A80' 
                THEN 'Other_order_type'
              ELSE ''
              END AS Type,

        CASE WHEN adopter IS NULL 
              THEN 'Other or not stated'
              ELSE adopter
              END AS adopter_2

FROM fcsq.adopt_disposals_with_child disposals_with_child
WHERE (disposals_with_child.event_model<>'A81') 
ORDER BY disposals_with_child.receipt_date;
"""

pydb.read_sql_query(create_adopt_disposals5)



#### adopt_disposals5 validation

In [ ]:
adopt_disposals5_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_disposals5")
adopt_disposals5_count

## adopt_disposals5_adoption table

### Drop the adopt_disposals5_adoption table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adopt_disposals5_adoption = f"""
DROP TABLE IF EXISTS fcsq.adopt_disposals5_adoption;
"""
pydb.read_sql_query(drop_adopt_disposals5_adoption)

# clean up previous adopt_disposals5_adoption files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_disposals5_adoption/").delete()

### Create the adopt_disposals5_adoption table in Athena

In [ ]:
create_adopt_disposals5_adoption = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_disposals5_adoption
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_disposals5_adoption') AS
SELECT disposals5.case_number, 
        disposals5.court, 
        disposals5.year, 
        disposals5.quarter, 
        disposals5.receipt_date, 
        disposals5.event_model, 
        disposals5.country_of_birth, 
        disposals5.number_applicants, 
        disposals5.adopter_type, 
        disposals5.adopter, 

        CASE WHEN child_sex=1 THEN'M'
              WHEN child_sex=2 THEN'F'
              ELSE 'U'
              END AS ch_sex, 

        disposals5.age_band, 
        disposals5.adoption, 
        disposals5.child_sex, 
        disposals5.type, 
        disposals5.adopter_2 

FROM fcsq.adopt_disposals5 disposals5
WHERE disposals5.adoption='Adoption';
"""

pydb.read_sql_query(create_adopt_disposals5_adoption)



#### adopt_disposals5_adoption validation

In [ ]:
adopt_disposals5_adoption_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_disposals5_adoption")
adopt_disposals5_adoption_count

## adopt_disposals5_non_adoption table

### Drop the adopt_disposals5_non_adoption table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adopt_disposals5_non_adoption = f"""
DROP TABLE IF EXISTS fcsq.adopt_disposals5_non_adoption;
"""
pydb.read_sql_query(drop_adopt_disposals5_non_adoption)

# clean up previous adopt_disposals5_non_adoption files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_disposals5_non_adoption/").delete()

### Create the adopt_disposals5_non_adoption table in Athena

In [ ]:
create_adopt_disposals5_non_adoption = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_disposals5_non_adoption
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_disposals5_non_adoption') AS
SELECT disposals5.case_number, 
        disposals5.court, 
        disposals5.year, 
        disposals5.quarter, 
        disposals5.receipt_date,
        disposals5.event_model, 
        disposals5.order_type, 
        disposals5.adoption, 
        disposals5.type, 
        disposals5.adopter_2,

        CASE WHEN type='Placement' THEN type
              ELSE 'Other Order'
              END  placement

FROM fcsq.adopt_disposals5 disposals5
WHERE (disposals5.event_model<>'A81') 
  AND (disposals5.adoption='Non-adoption');
"""

pydb.read_sql_query(create_adopt_disposals5_non_adoption)



#### adopt_disposals5_non_adoption validation

In [ ]:
adopt_disposals5_non_adoption_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_disposals5_non_adoption")
adopt_disposals5_non_adoption_count

## adopt_disposals5_non_orders table

### Drop the adopt_disposals5_non_orders table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adopt_disposals5_non_orders = f"""
DROP TABLE IF EXISTS fcsq.adopt_disposals5_non_orders;
"""
pydb.read_sql_query(drop_adopt_disposals5_non_orders)

# clean up previous adopt_disposals5_non_orders files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_disposals5_non_orders/").delete()

### Create the adopt_disposals5_non_orders table in Athena

In [ ]:
create_adopt_disposals5_non_orders = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_disposals5_non_orders
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_disposals5_non_orders') AS
SELECT disposals5.case_number, 
        disposals5.court, 
        disposals5.year, 
        disposals5.quarter, 
        disposals5.receipt_date, 
        disposals5.event_model,
        disposals5.adoption, 
        disposals5.adopter_2 
FROM fcsq.adopt_disposals5 disposals5
WHERE disposals5.adoption='No order made';
"""

pydb.read_sql_query(create_adopt_disposals5_non_orders)



#### adopt_disposals5_non_orders validation

In [ ]:
adopt_disposals5_non_orders_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_disposals5_non_orders")
adopt_disposals5_non_orders_count

In [ ]:
#Summary 4
summary4 = f"""
SELECT year, 
        quarter, 
        SUM(CASE WHEN adopter_2 ='Other or not stated' THEN 1 ELSE 0 END) other_or_not_stated,
        SUM(CASE WHEN adopter_2 ='Sole applicant' THEN 1 ELSE 0 END) sole_applicant,
        SUM(CASE WHEN adopter_2 ='Step parent' THEN 1 ELSE 0 END) step_parent,
        SUM(CASE WHEN adopter_2 ='mixed-sex couple' THEN 1 ELSE 0 END) mixed_sex_couple,
        SUM(CASE WHEN adopter_2 ='same-sex couple' THEN 1 ELSE 0 END) same_sex_couple,
        COUNT(*) adopter_2_total
from fcsq.adopt_disposals5_adoption
where year > 2010
GROUP BY year, 
          quarter
ORDER BY year,
          quarter;
"""

pydb.read_sql_query(summary4)


In [ ]:
summary5 = f"""
SELECT year, 
        quarter, 
        COUNT (case_number) AS count_of_case

FROM fcsq.adopt_disposals5_adoption
GROUP BY year, quarter
ORDER BY year, quarter;
"""
pydb.read_sql_query(summary5)

In [ ]:
summary6 = f"""
SELECT year, 
        quarter,
        SUM(CASE WHEN ch_sex='F' THEN 1 ELSE 0 END) f,
        SUM(CASE WHEN ch_sex='M' THEN 1 ELSE 0 END) m,
        SUM(CASE WHEN ch_sex='U' THEN 1 ELSE 0 END) u,
        COUNT(*) total
from adopt_disposals5_adoption
WHERE year > 2010
GROUP BY year, quarter
ORDER BY year,quarter;
"""
pydb.read_sql_query(summary6)

In [ ]:
summary7 = f"""
SELECT year, 
        quarter,
        SUM(CASE WHEN age_band ='<1 year' THEN 1 ELSE 0 END) less_1_year,
        SUM(CASE WHEN age_band='1-4 years' THEN 1 ELSE 0 END) years_1_4,
        SUM(CASE WHEN age_band ='5-9 years' THEN 1 ELSE 0 END) years_5_9,
        SUM(CASE WHEN age_band ='10-14 years' THEN 1 ELSE 0 END) years_10_14,
        SUM(CASE WHEN age_band ='15-17 years' THEN 1 ELSE 0 END) year_15_17,
        SUM(CASE WHEN age_band = 'Other' THEN 1 ELSE 0 END) other,
        SUM(CASE WHEN age_band = 'Unknown' THEN 1 ELSE 0 END) unknown,
        COUNT(*) total
FROM adopt_disposals5_adoption
WHERE year > 2010
GROUP BY year, quarter
ORDER BY year,quarter;

"""
pydb.read_sql_query(summary7)

In [ ]:
summary8 = f"""
SELECT year, 
        quarter,
        SUM(CASE WHEN placement ='Other Order' THEN 1 ELSE 0 END) other_order,
        SUM(CASE WHEN placement ='Placement' THEN 1 ELSE 0 END) placement,
        COUNT(*) total
FROM  adopt_disposals5_non_adoption
WHERE year>2010
GROUP BY year, quarter
ORDER BY year, quarter;
"""
pydb.read_sql_query(summary8)

In [ ]:
summary9 = f"""
SELECT year,
        quarter, 
        COUNT(*) total_non_orders
FROM adopt_disposals5_non_orders
WHERE year>2010
GROUP BY year, quarter
ORDER BY year, quarter;

"""
pydb.read_sql_query(summary9)

In [ ]:
#Summary 10: of disposal case counts


create_adopt_disposal_cases = f"""
SELECT disposals5.case_number, 
        MIN(disposals5.receipt_date) AS minofreceipt_date 
FROM fcsq.adopt_disposals5 disposals5
GROUP BY disposals5.case_number;
"""

pydb.create_temp_table(create_adopt_disposal_cases, "adopt_disposal_cases")

create_adopt_disposal_cases_b = f"""
SELECT disposal_cases.minofreceipt_date,
        disposal_cases.case_number,
        cast(EXTRACT(YEAR FROM (disposal_cases.minofreceipt_date)) as varchar(3)) ||' - '||

        CASE WHEN EXTRACT(MONTH FROM (disposal_cases.minofreceipt_date))<4 THEN '1'
              WHEN EXTRACT(MONTH FROM (disposal_cases.minofreceipt_date))<7 THEN '2'
              WHEN EXTRACT(MONTH FROM (disposal_cases.minofreceipt_date))<10 THEN '3'
              ELSE '4'
              END AS quarter

FROM __temp__.adopt_disposal_cases disposal_cases;
"""
pydb.create_temp_table(create_adopt_disposal_cases_b, "adopt_disposal_cases_b")

create_adopt_disp_case_count_total ="""
SELECT quarter, 
        COUNT(case_number) as count_of_case
FROM __temp__.adopt_disposal_cases_b
GROUP BY  quarter;

"""
pydb.read_sql_query(create_adopt_disp_case_count_total)


## adopt_disposals5_2011 table

### Drop the adopt_disposals5_2011 table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adopt_disposals5_2011 = f"""
DROP TABLE IF EXISTS fcsq.adopt_disposals5_2011;
"""
pydb.read_sql_query(drop_adopt_disposals5_2011)

# clean up previous adopt_disposals5_2011 files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_disposals5_2011/").delete()

### Create the adopt_disposals5_2011 table in Athena

In [ ]:
create_adopt_disposals5_2011 = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_disposals5_2011
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_disposals5_2011') AS
SELECT t1.*
FROM fcsq.adopt_Disposals5 t1
WHERE year >2010;
"""

pydb.read_sql_query(create_adopt_disposals5_2011)



#### adopt_disposals5_2011 validation

In [ ]:
adopt_disposals5_2011_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_disposals5_2011")
adopt_disposals5_2011_count

In [ ]:
"""
/* The next couple of queries are here to find the court level data for the 
court level/DFJ file. We do not use the below for FCSQ anymore - table names will be wrong*/

/*Query 18
Firstly the disposals court level file*/

/*

select year, 
        quarter, 
        court, 
        count(*) court_Total
from Adoptions_Disposals5_adoption
where year>2010
group by year, quarter, court
order by year, quarter, court;
 	
/*Secondly the applications case level court level file*/

/*
drop table adopt_Disposal_case_count_A; 
Create table adopt_Disposal_case_count_A AS

SELECT t1.CASE_NUMBER, 
        Min(t1.Receipt_date) AS MinOfReceipt_date,
        t1.Court 
FROM adopt_Disposals5 t1 
GROUP BY t1.CASE_NUMBER, t1.Court;

drop table adopt_Disposal_case_count_B;
Create table adopt_Disposal_case_count_B AS

SELECT CASE_NUMBER, 
        extract (Year from (MinOfReceipt_date)) AS YEAR,

        CASE WHEN extract(Month from(MinOfReceipt_date))<4 THEN 1
              WHEN extract(Month from(MinOfReceipt_date))<7 THEN 2
              WHEN extract(Month from(MinOfReceipt_date))<10 THEN 3
              ELSE 4
              END AS Quarter,
        
        MinOfReceipt_date,
        Court 
        
FROM adopt_Disposal_case_count_A;
"""

# Stage 4 - Now to update the country of birth lookup

In [ ]:
#Create parquet file to add into athena for birth_country_lookup table
data = pd.read_csv('s3://alpha-family-data/fcsq_processing/Adoption/birth_country_lookup_csv/Adopt_birth_country_lookup.csv')
import os
os.environ['AWS_DEFAULT_REGION'] = 'eu-west-2'
import awswrangler as wr
wr.s3.to_parquet(  # Storing the data and metadata to Data Lake
    df=data,
    path="s3://alpha-family-data/fcsq_processing/Adoption/birth_country_lookup_csv/birth_country_lookup_parquet/",
    dataset=True, database = "default", table = "birth_country_lookup"
)



In [ ]:
#Create parquet file to add into athena for event_model_desc table
data = pd.read_csv('s3://alpha-family-data/fcsq_processing/Adoption/event_model_desc_csv/Event_model_desc.csv')
import os
os.environ['AWS_DEFAULT_REGION'] = 'eu-west-2'
import awswrangler as wr
wr.s3.to_parquet(  # Storing the data and metadata to Data Lake
    df=data,
    path="s3://alpha-family-data/fcsq_processing/Adoption/event_model_desc_csv/Event_model_desc_parquet/",
    dataset=True, database = "default", table = "adoption_event_model_desc"
)


## adopt_birth_country_unknown table

### Create the adopt_birth_country_unknown temp table

In [ ]:
create_adopt_birth_country_unknown = f"""
SELECT DISTINCT
        a.country_of_birth,
        b.nationality
FROM fcsq.adopt_disposals5_2011 a

LEFT JOIN fcsq.adopt_birth_country_lookup b
  ON a.country_of_birth = b.country_of_birth
  
WHERE a.country_of_birth IS NOT NULL
  AND b.nationality IS NULL;
"""

pydb.create_temp_table(create_adopt_birth_country_unknown,"adopt_birth_country_unknown")



#### adopt_birth_country_unknown validation

In [ ]:
adopt_birth_country_unknown_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_birth_country_unknown")
adopt_birth_country_unknown_count

## adopt_birth_country_added table

### Create the adopt_birth_country_added temp table

In [ ]:
create_adopt_birth_country_added = f"""
SELECT country_of_birth,
        CASE WHEN country_of_birth IN ('lithunia', 'republic of the philipines', 'united states of america (the city of new york)', 'morroco', 'saint lucia') /*update countries listed according to list in 'adopt_birth_country_unknown'*/
                THEN 2 /* For foreign countries only */
              /*WHEN country_of_birth in ('...')
                THEN 1*/ /*UK countries only - add to when necessary*/
              END AS nationality

FROM __temp__.adopt_birth_country_unknown;
"""

pydb.create_temp_table(create_adopt_birth_country_added,'adopt_birth_country_added')



#### adopt_birth_country_added validation

In [ ]:
adopt_birth_country_added_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_birth_country_added")
adopt_birth_country_added_count

## adopt_country_lookup_update table

### Create the adopt_country_lookup_update temp table

In [ ]:
create_adopt_country_lookup_update = f"""
SELECT country_of_birth, nationality      /* To account for the columns being in a different order from the '_added' table */
FROM fcsq.adopt_birth_country_lookup

UNION
SELECT *
FROM __temp__.adopt_birth_country_added

ORDER BY country_of_birth;

"""

pydb.create_temp_table(create_adopt_country_lookup_update,'adopt_country_lookup_update')



#### adopt_country_lookup_update validation

In [ ]:
adopt_country_lookup_update_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_country_lookup_update")
adopt_country_lookup_update_count

## adopt_birth_country_lookup table

### Drop the adopt_birth_country_lookup table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adopt_birth_country_lookup = f"""
DROP TABLE IF EXISTS fcsq.adopt_birth_country_lookup;
"""
pydb.read_sql_query(drop_adopt_birth_country_lookup)

# clean up previous adopt_birth_country_lookup files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_birth_country_lookup/").delete()

### Update the adopt_birth_country_lookup table in Athena

In [ ]:
create_adopt_birth_country_lookup = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_birth_country_lookup
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_birth_country_lookup') AS
SELECT *
FROM __temp__.adopt_country_lookup_update;
"""

pydb.read_sql_query(create_adopt_birth_country_lookup)



#### adopt_birth_country_lookup validation

In [ ]:
adopt_birth_country_lookup_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_birth_country_lookup")
adopt_birth_country_lookup_count

# Stage 5 - Now to prepare the final output

## adopt_adoptions_only_standard table

### Create the adopt_adoptions_only_standard temp table

In [ ]:
create_adopt_adoptions_only_standard = f"""
SELECT type,
        year,
        quarter,
        application,
        order_type,
        adopter_3,
        COUNT(*) AS count
FROM(

SELECT
  'Application' as type,
  year,
  quarter,
  adoption_cases AS application, /*see where clause below*/
  'Standard' AS order_type,
  CASE WHEN adopter_2 IN ('','Other')
         THEN 'Other or not stated'
	   WHEN adopter_2 = 'mixed-sex couple'	
         THEN 'M/F couple'
       WHEN adopter_2 = 'same-sex couple'
         THEN 'Same sex couple'
	 ELSE adopter_2
    END AS adopter_3
FROM
  fcsq.adopt_apps_6_adoptions_only
WHERE
  standard = 1 /*and adoption_cases = 'Adoption'*//*Adopt+other and foreign use adoption, standard and convention use adoption cases (copied from excel process*/
)

GROUP BY
  type,
  year,
  quarter,
  application,
  order_type,
  adopter_3;
"""

pydb.create_temp_table(create_adopt_adoptions_only_standard,"adopt_adoptions_only_standard")



#### adopt_adoptions_only_standard validation

In [ ]:
adopt_adoptions_only_standard_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_adoptions_only_standard")
adopt_adoptions_only_standard_count

## adopt_adoption_only_convention table

### Create the adopt_adoption_only_convention temp table

In [ ]:
create_adopt_adoption_only_convention = f"""
SELECT type,
        year,
        quarter,
        application,
        order_type,
        adopter_3,
        COUNT(*) AS count
FROM(
  
SELECT
  'Application' as type,
  year,
  quarter,
  adoption_cases AS application, /*see where clause below*/
  'Convention' AS order_type,
  CASE WHEN adopter_2 IN ('','Other')
         THEN 'Other or not stated'
	   WHEN adopter_2 = 'mixed-sex couple'	
         THEN 'M/F couple'
       WHEN adopter_2 = 'same-sex couple'
         THEN 'Same sex couple'
	 ELSE adopter_2
    END AS adopter_3
FROM
  fcsq.adopt_apps_6_adoptions_only
WHERE
  convention = 1 /*and adoption_cases = 'Adoption'*//*Adopt+other and foreign use adoption, standard and convention use adoption cases (copied from excel process*/
)

GROUP BY
  type,
  year,
  quarter,
  application,
  order_type,
  adopter_3;
"""

pydb.create_temp_table(create_adopt_adoption_only_convention,'adopt_adoption_only_convention')



#### adopt_adoption_only_convention validation

In [ ]:
adopt_adoption_only_convention_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_adoption_only_convention")
adopt_adoption_only_convention_count

## adopt_adoptions_only_foreign table

### Create the adopt_adoptions_only_foreign temp table

In [ ]:
create_adopt_adoptions_only_foreign = f"""
SELECT type,
        year,
        quarter,
        application,
        order_type,
        adopter_3,
        COUNT(*) AS count
FROM(  
  
SELECT
  'Application' as type,
  year,
  quarter,
  adoption AS application, /*see where clause below*/
  'Foreign' AS order_type,
  CASE WHEN adopter_2 IN ('','Other')
         THEN 'Other or not stated'
	   WHEN adopter_2 = 'mixed-sex couple'	
         THEN 'M/F couple'
       WHEN adopter_2 = 'same-sex couple'
         THEN 'Same sex couple'
	 ELSE adopter_2
    END AS adopter_3
FROM
  fcsq.adopt_apps_6_adoptions_only
WHERE
  foreign = 1 and adoption = 'Adoption'/*Adopt+other and foreign use adoption, standard and convention use adoption cases (copied from excel process*/
)  
  
GROUP BY
  type,
  year,
  quarter,
  application,
  order_type,
  adopter_3;
"""

pydb.create_temp_table(create_adopt_adoptions_only_foreign,'adopt_adoptions_only_foreign')



#### adopt_adoptions_only_foreign validation

In [ ]:
adopt_adoptions_only_foreign_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_adoptions_only_foreign")
adopt_adoptions_only_foreign_count

## adopt_adoptions_other_only table

### Create the adopt_adoptions_other_only temp table

In [ ]:
create_adopt_adoptions_other_only = f"""
SELECT type,
        year,
        quarter,
        application,
        order_type,
        adopter_3,
        COUNT(*) AS COUNT
FROM(  
  
SELECT
  'Application' as type,
  year,
  quarter,
  adoption AS application, /*see where clause below*/
  'Foreign' AS order_type,
  CASE WHEN adopter_2 IN ('','Other')
         THEN 'Other or not stated'
	   WHEN adopter_2 = 'mixed-sex couple'	
         THEN 'M/F couple'
       WHEN adopter_2 = 'same-sex couple'
         THEN 'Same sex couple'
	 ELSE adopter_2
    END AS adopter_3
FROM
  fcsq.adopt_apps_6_adoptions_only
WHERE
  foreign = 1 AND adoption = 'Adoption+other'/*Adopt+other and foreign use adoption, standard and convention use adoption cases (copied from excel process*/
)

GROUP BY
        type,
        year,
        quarter,
        application,
        order_type,
        adopter_3;
"""

pydb.create_temp_table(create_adopt_adoptions_other_only,'adopt_adoptions_other_only')



#### adopt_adoptions_other_only validation

In [ ]:
adopt_adoptions_other_only_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_adoptions_other_only")
adopt_adoptions_other_only_count

## adopt_adoption_only table

### Create the adopt_adoption_only temp table

In [ ]:
create_adopt_adoption_only = f"""
SELECT
  *
FROM
  __temp__.adopt_adoptions_only_standard
UNION ALL 
SELECT
  *
FROM
  __temp__.adopt_adoption_only_convention
UNION ALL 
SELECT
  *
FROM
  __temp__.adopt_adoptions_only_foreign
UNION ALL 
SELECT
  *
FROM
  __temp__.adopt_adoptions_other_only;
"""

pydb.create_temp_table(create_adopt_adoption_only,'adopt_adoption_only')



#### adopt_adoption_only validation

In [ ]:
adopt_adoption_only_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_adoption_only")
adopt_adoption_only_count

## adopt_adoption_placement table

### Create the adopt_adoption_placement temp table

In [ ]:
create_adopt_adoption_placement = f"""
SELECT
  'Application' AS type,
  year,
  quarter,
  'Non-adoption' as application,
  'Placement' AS order_type,
  'n/a' AS adopter,
   placement AS count
FROM
 fcsq.adopt_apps_6_non_adoptions
WHERE
  placement > 0;

"""

pydb.create_temp_table(create_adopt_adoption_placement,'adopt_adoption_placement')



#### adopt_adoption_placement validation

In [ ]:
adopt_adoption_placement_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_adoption_placement")
adopt_adoption_placement_count

## adopt_placement_revoke_or_vary table

### Create the adopt_placement_revoke_or_vary temp table

In [ ]:
create_adopt_placement_revoke_or_vary = f"""
SELECT
  'Application' AS type,
  year,
  quarter,
  'Non-adoption' as application,
  'Placement_revoke_or_vary' AS order_type,
  'n/a' AS adopter,
  placement_revoke_or_vary AS count
FROM
  fcsq.adopt_apps_6_non_adoptions
WHERE
  placement_revoke_or_vary > 0;
"""

pydb.create_temp_table(create_adopt_placement_revoke_or_vary,'adopt_placement_revoke_or_vary')



#### adopt_placement_revoke_or_vary validation

In [ ]:
adopt_placement_revoke_or_vary_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_placement_revoke_or_vary")
adopt_placement_revoke_or_vary_count

## adopt_adoption_contact_s26 table

### Create the adopt_adoption_contact_s26 temp table

In [ ]:
create_adopt_adoption_contact_s26 = f"""
SELECT
  'Application' AS type,
  year,
  quarter,
  'Non-adoption' as application,
  'Contact_s26' AS order_type,
  'n/a' AS adopter,
  contact_s26 AS count
FROM
  fcsq.adopt_apps_6_non_adoptions
WHERE
  contact_s26 > 0;
"""

pydb.create_temp_table(create_adopt_adoption_contact_s26,'adopt_adoption_contact_s26')



#### adopt_adoption_contact_s26 validation

In [ ]:
adopt_adoption_contact_s26_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_adoption_contact_s26")
adopt_adoption_contact_s26_count

## adopt_contact_s26_revoke_vary table

### Create the adopt_contact_s26_revoke_vary temp table

In [ ]:
create_adopt_contact_s26_revoke_vary = f"""
SELECT
  'Application' AS type,
  year,
  quarter,
  'Non-adoption' as application,
  'Contact_s26_revoke_or_vary' AS order_type,
  'n/a' AS adopter,
  contact_s26_revoke_or_vary AS count
FROM
  fcsq.adopt_apps_6_non_adoptions
WHERE
  contact_s26_revoke_or_vary > 0;
"""

pydb.create_temp_table(create_adopt_contact_s26_revoke_vary,'adopt_contact_s26_revoke_vary')



#### adopt_contact_s26_revoke_vary validation

In [ ]:
adopt_contact_s26_revoke_vary_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_contact_s26_revoke_vary")
adopt_contact_s26_revoke_vary_count

## adopt_change_surname table

### Create the adopt_change_surname temp table

In [ ]:
create_adopt_change_surname = f"""
SELECT
  'Application' AS type,
  year,
  quarter,
  'Non-adoption' as application,
  'Change_surname' AS order_type,
  'n/a' AS adopter,
  change_surname AS count
FROM
  fcsq.adopt_apps_6_non_adoptions
WHERE
  change_surname > 0;
"""

pydb.create_temp_table(create_adopt_change_surname,'adopt_change_surname')



#### adopt_change_surname validation

In [ ]:
adopt_change_surname_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_change_surname")
adopt_change_surname_count

## adopt_remove_child_from_uk table

### Create the adopt_remove_child_from_uk temp table

In [ ]:
create_adopt_remove_child_from_uk = f"""
SELECT
  'Application' AS type,
  year,
  quarter,
  'Non-adoption' as application,
  'Remove_child_from_UK' AS order_type,
  'n/a' AS adopter,
  remove_child_from_uk AS count
FROM
  fcsq.adopt_apps_6_non_adoptions
WHERE
  remove_child_from_uk > 0;
"""

pydb.create_temp_table(create_adopt_remove_child_from_uk,'adopt_remove_child_from_uk')



#### adopt_remove_child_from_uk validation

In [ ]:
adopt_remove_child_from_uk_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_remove_child_from_uk")
adopt_remove_child_from_uk_count

## adopt_other_order_type table

### Create the adopt_other_order_type temp table

In [ ]:
create_adopt_other_order_type = f"""
SELECT
  'Application' AS type,
  year,
  quarter,
  'Non-adoption' as application,
  'Other_order_type' AS order_type,
  'n/a' AS adopter,
  other_order_type AS count
FROM
  fcsq.adopt_apps_6_non_adoptions
WHERE
  other_order_type > 0;
"""

pydb.create_temp_table(create_adopt_other_order_type,'adopt_other_order_type')



#### adopt_other_order_type validation

In [ ]:
adopt_other_order_type_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_other_order_type")
adopt_other_order_type_count

## adopt_non_adopt_types table

### Create the adopt_non_adopt_types temp table

In [ ]:
create_adopt_non_adopt_types = f"""
SELECT
  *
FROM
  __temp__.adopt_adoption_placement
UNION ALL
SELECT
  *
FROM
  __temp__.adopt_placement_revoke_or_vary
UNION ALL
SELECT
  *
FROM
  __temp__.adopt_adoption_contact_s26
UNION ALL
SELECT
  *
FROM
  __temp__.adopt_contact_s26_revoke_vary
UNION ALL
SELECT
  *
FROM
  __temp__.adopt_change_surname
UNION ALL
SELECT
  *
FROM
  __temp__.adopt_remove_child_from_uk
UNION ALL
SELECT
  *
FROM
  __temp__.adopt_other_order_type;
"""

pydb.create_temp_table(create_adopt_non_adopt_types,'adopt_non_adopt_types')



#### adopt_non_adopt_types validation

In [ ]:
adopt_non_adopt_types_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_non_adopt_types")
adopt_non_adopt_types_count

## adopt_non_adoptions table

### Create the adopt_non_adoptions temp table

In [ ]:
create_adopt_non_adoptions = f"""
SELECT
  type,
  year,
  quarter,
  application,
  order_type,
  adopter,
  SUM (count) AS count
FROM
  __temp__.adopt_non_adopt_types
GROUP BY
  type,
  year,
  quarter,
  application,
  order_type,
  adopter;
"""

pydb.create_temp_table(create_adopt_non_adoptions,'adopt_non_adoptions')



#### adopt_non_adoptions validation

In [ ]:
adopt_non_adoptions_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_non_adoptions")
adopt_non_adoptions_count

## adopt_applications table

### Create the adopt_applications temp table

In [ ]:
create_adopt_applications = f"""

SELECT
  type,
  year,
  quarter,
  application,
  order_type,
  adopter_3 AS adopter,
  'n/a' as adopted_child_sex,	
  'n/a' as adopted_child_age,
  count
FROM
  __temp__.adopt_adoption_only
UNION ALL
SELECT
  type,
  year,
  quarter,
  application,
  order_type,
  adopter,
  'n/a' as adopted_child_sex,	
  'n/a' as adopted_child_age,
  count
FROM
  __temp__.adopt_non_adoptions;

"""

pydb.create_temp_table(create_adopt_applications,'adopt_applications')



#### adopt_applications validation

In [ ]:
adopt_applications_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_applications")
adopt_applications_count

# Disposals

## adopt_disp_details table

### Create the adopt_disp_details temp table

In [ ]:
create_adopt_disp_details = f"""
SELECT
  a.*,
  CASE WHEN b.nationality = 2
    THEN 'Foreign'
	ELSE 'N'
  END AS foreign,
  c.event_model_desc
FROM
  fcsq.adopt_disposals5_2011 a
   LEFT JOIN fcsq.adopt_birth_country_lookup b
   ON a.country_of_birth = b.country_of_birth
   LEFT JOIN fcsq.adoption_event_model_desc c
    ON a.event_model = c.event_model;
"""

pydb.create_temp_table(create_adopt_disp_details,'adopt_disp_details')



#### adopt_disp_details validation

In [ ]:
adopt_disp_details_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_disp_details")
adopt_disp_details_count

## adopt_disp_csv_data table

### Create the adopt_disp_csv_data temp table

In [ ]:
create_adopt_disp_csv_data = f"""
SELECT
  CASE WHEN adoption IN ('Adoption', 'Non-adoption') THEN 'Order granted'
       WHEN adoption = 'No order made' THEN 'No order granted'
     ELSE 'Check'
    END AS type,
  year,
  quarter,
  adoption,
  CASE WHEN adoption = 'Adoption'
         AND event_model = 'A77'
	  THEN 'Convention'
       WHEN adoption = 'Adoption'
         AND foreign = 'Foreign'
	  THEN 'Foreign'
       WHEN adoption = 'Adoption'
      THEN 'Standard'
	 ELSE type
    END AS type_of_adoption,
  event_model_desc,
  CASE WHEN adoption IN ('No order made','Non-adoption')
         THEN 'n/a'
       WHEN adopter_2 IN ('','Other')
         THEN 'Other or not stated'
	   WHEN adopter_2 = 'mixed-sex couple'	
         THEN 'M/F couple'
       WHEN adopter_2 = 'same-sex couple'
         THEN 'Same sex couple'
	 ELSE adopter_2
	END AS adopter,
  CASE WHEN adoption IN ('No order made','Non-adoption')
         THEN 'n/a'
       WHEN child_sex = 1 THEN 'Male'
       WHEN child_sex = 2 THEN 'Female'
	   ELSE 'Unknown'
	 END AS adopted_child_sex,
  CASE WHEN adoption IN ('No order made','Non-adoption')
         THEN 'n/a'
       WHEN age_band IN ('<1 year', '10-14 years', '1-4 years', '15-17 years', '5-9 years', 'Unknown')
        THEN age_band
	  ELSE 'Other'
   END AS adopted_child_age
 FROM
   __temp__.adopt_disp_details;
"""

pydb.create_temp_table(create_adopt_disp_csv_data,'adopt_disp_csv_data')



#### adopt_disp_csv_data validation

In [ ]:
adopt_disp_csv_data_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_disp_csv_data")
adopt_disp_csv_data_count

## adopt_disp_order_type table

### Create the adopt_disp_order_type temp table

In [ ]:
create_adopt_disp_order_type = f"""
SELECT
  a.*,
  CASE WHEN a.adoption IN ('Adoption', 'Non-adoption')
      THEN type_of_adoption
	   WHEN a.adoption = 'No order made'
      THEN event_model_desc
	 END AS order_type
FROM
  __temp__.adopt_disp_csv_data a;
"""

pydb.create_temp_table(create_adopt_disp_order_type,'adopt_disp_order_type')



#### adopt_disp_order_type validation

In [ ]:
adopt_disp_order_type_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_disp_order_type")
adopt_disp_order_type_count

## adopt_disposals table

### Create the adopt_disposals temp table

In [ ]:
create_adopt_disposals = f"""
SELECT
  type,
  year,
  quarter,
  adoption,
  order_type,
  adopter,
  adopted_child_sex,
  adopted_child_age,
  COUNT (*) AS count
FROM
  __temp__.adopt_disp_order_type
GROUP BY
  type,
  year,
  quarter,
  adoption,
  order_type,
  adopter,
  adopted_child_sex,
  adopted_child_age;
"""

pydb.create_temp_table(create_adopt_disposals,'adopt_disposals')



#### adopt_disposals validation

In [ ]:
adopt_disposals_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_disposals")
adopt_disposals_count

# Cases

## adopt_application_case_count table

### Create the adopt_application_case_count temp table

In [ ]:
adopt_application_case_count_a = f"""
SELECT case_number,
        court, 
        MIN(app_date) AS minofapp_date
FROM fcsq.adopt_application_4 
WHERE year > 2010
GROUP BY case_number,court
ORDER BY 1,2;
"""
pydb.create_temp_table(adopt_application_case_count_a,"adopt_application_case_count_a")

In [ ]:
create_adopt_application_case_count_table= f"""
SELECT type,
        year,
        quarter,
        adoption,
        order_type,
        adopter,
        adopted_child_sex,
        adopted_child_age,
        COUNT(*) AS count
FROM(        

SELECT 
  'Cases started' AS type,
  EXTRACT(YEAR FROM minofapp_date) year,

  CASE WHEN  Minofapp_date is null THEN null
        WHEN EXTRACT(MONTH FROM minofapp_date) BETWEEN 1 AND 3 THEN 1
        WHEN EXTRACT(MONTH FROM minofapp_date) BETWEEN 4 AND 6 THEN 2
        WHEN EXTRACT(MONTH FROM minofapp_date) BETWEEN 7 AND 9 THEN 3
        WHEN EXTRACT(MONTH FROM minofapp_date) BETWEEN 10 AND 12 THEN 4
        END quarter,
  
  'n/a' AS adoption,
  'n/a' AS order_type,
  'n/a' AS adopter,
  'n/a' AS adopted_child_sex,
  'n/a' AS adopted_child_age
FROM 
  __temp__.adopt_application_case_count_a
)  
  
GROUP BY
  type,
  year,
  quarter,
  adoption,
  order_type,
  adopter,
  adopted_child_sex,
  adopted_child_age;
"""
#table1 = pydb.read_sql_query(create_adopt_application_case_count)
pydb.create_temp_table(create_adopt_application_case_count_table,"adopt_application_case_count_table")


#### adopt_application_case_count validation

In [ ]:
adopt_application_case_count_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_application_case_count_table")
adopt_application_case_count_count

## adopt_disposal_case_count table

### Create the adopt_disposal_case_count temp table

In [ ]:
create_adopt_disposal_case_count = f"""
SELECT type,
        year,
        quarter,
        adoption,
        order_type,
        adopter,
        adopted_child_sex,
        adopted_child_age,
        COUNT(*) AS count
FROM( 

SELECT 
  'Cases closed' as type,
  EXTRACT(YEAR FROM minofreceipt_date) year,

  CASE WHEN  minofreceipt_date IS NULL THEN NULL
        WHEN EXTRACT(Month FROM minofreceipt_date) BETWEEN 1 AND 3 THEN 1
        WHEN EXTRACT(Month FROM minofreceipt_date) BETWEEN 4 AND 6 THEN 2
        WHEN EXTRACT(Month FROM minofreceipt_date) BETWEEN 7 AND 9 THEN 3
        WHEN EXTRACT(Month FROM minofreceipt_date) BETWEEN 10 AND 12 THEN 4
        END quarter,
  'n/a' AS adoption,
  'n/a' AS order_type,
  'n/a' AS adopter,
  'n/a' AS adopted_child_sex,
  'n/a' AS adopted_child_age
FROM 
  __temp__.adopt_disposal_cases
WHERE
  (EXTRACT(YEAR FROM minofreceipt_date)) > 2010
)  
  
GROUP BY
  type,
  year,
  quarter,
  adoption,
  order_type,
  adopter,
  adopted_child_sex,
  adopted_child_age;
"""

pydb.create_temp_table(create_adopt_disposal_case_count,'adopt_disposal_case_count')



#### adopt_disposal_case_count validation

In [ ]:
adopt_disposal_case_count_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_disposal_case_count")
adopt_disposal_case_count_count

## adopt_csv table

### Create the adopt_csv temp table

In [ ]:
create_adopt_csv = f"""
SELECT
  *
FROM
  __temp__.adopt_applications
UNION ALL
SELECT
  *
FROM
  __temp__.adopt_disposals
UNION ALL
SELECT
  *
FROM
  __temp__.adopt_application_case_count
UNION ALL
SELECT
  *
FROM
  __temp__.adopt_disposal_case_count;
"""

pydb.create_temp_table(create_adopt_csv,'adopt_csv')



#### adopt_csv validation

In [ ]:
adopt_csv_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_csv")
adopt_csv_count

## adopt_csv2 table

### Drop the adopt_csv2 table if it already exists and remove its data from the S3 bucket

In [ ]:
drop_adopt_csv2 = f"""
DROP TABLE IF EXISTS fcsq.adopt_csv2;
"""
pydb.read_sql_query(drop_adopt_csv2)

# clean up previous adopt_csv2 files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_csv2/").delete()

### Create the adopt_csv2 table in Athena

In [ ]:
create_adopt_csv2 = f"""
SELECT
  *
FROM
  __temp__.adopt_csv
WHERE
  year > 2010
  AND NOT (year = 2021 AND quarter = 3)
ORDER BY
  type,
  application,
  order_type,
  year,
  quarter,
  adopter,
  adopted_child_sex,
  adopted_child_age;
"""

table = pydb.read_sql_query(create_adopt_csv2)
#table.to_csv(path_or_buf = '~/FCSQ_data/adopt_csv2.csv')
table

In [ ]:
adopt_csv2_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_csv2")
adopt_csv2_count

In [ ]:
df = pandas.read_csv("~/FCSQ_data/adopt_csv_copy.csv")